# Data Preprocessing

## Preperation

Import packages and set globals

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:.2f}".format

%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 6)

Importing the pickle files

In [2]:
gdpDf = pd.read_pickle("../data/raw/gdp.pkl")
occDf = pd.read_pickle("../data/raw/naics_occupation.pkl")
patDf = pd.read_pickle("../data/raw/naics_pattern.pkl")

## Filtering

Filters are based on previous research within the DataFrames (Step 2)

Establish filter:

- NAIC
- Rough NAICS (for gdp)
- FIPS
- OCC

In [3]:
naics_filter = ["2381", "3311", "3330A1", "3361", "3363"]
naics_rough_filter = ["23", "321,327-339"]

fips_filter_inverse = ["0", "$999"]
fips_filter_inverse = "|".join(fips_filter_inverse)

occ_filter = ["51-4121", "51-4031", "51-4041", "49-9071", "51-2031"]

Apply filters

In [4]:
# NAICS
gdpDf = gdpDf[gdpDf["IndustryClassification"].isin(naics_rough_filter)]
occDf = occDf[occDf["naics"].isin(naics_filter)]
patDf = patDf[patDf["naics"].isin(naics_filter)]

# FIPS
gdpDf = gdpDf[~gdpDf["FIPS"].astype(str).str.contains(fips_filter_inverse)]
occDf = occDf[~occDf["FIPS"].astype(str).str.contains(fips_filter_inverse)]
patDf = patDf[~patDf["FIPS"].astype(str).str.contains(fips_filter_inverse)]

# OCC
occDf = occDf[occDf["OCC_CODE"].isin(occ_filter)]

## Drop Columns

The final "master Frame" should contain the following features:

- fips
- naics
- mean_gdp
- emp_total_county_naics
- emp_total_county_naics_top5_occ
- occ_code
- emp_occupation
- ap
- ap_top5_occ
- est

Establish arrays of features that should be dropped

In [5]:
cols_to_drop_gdp = ["GeoName", "Region", "TableName", "LineCode", "Description", "Unit"]
cols_to_drop_occ = ["State_GEOID", "NAICS_TITLE", "OCC_TITLE"]
cols_to_drop_pat = ["State_GEOID", "County_GEOID", "naics_2", "DESCRIPTION", "emp_nf", "emp", "qp1_nf", "qp1", "n<5", "n5_9", "n10_19", "n20_49", "n50_99", "n100_249", "n250_499", "n500_999", "n1000", "n1000_1", "n1000_2", "n1000_3", "n1000_4"]

In [6]:
gdpDf = gdpDf.drop(cols_to_drop_gdp, axis=1)
occDf = occDf.drop(cols_to_drop_occ, axis=1)
patDf = patDf.drop(cols_to_drop_pat, axis=1)

## New Features

Features that need to be created:

- mean_gdp
- IndustryClassification for occ/pattern to merge gdp
- emp_total_county_naics_top5_occ
    Total of employees in county for naics (based on our top 5 occupations)
- ap_top5_occ
    Broken down (AP/emp_total_county_naics) * emp_total_county_naics_top5_occ

And aggregate pattern duplicates

### Mean GDP

In [7]:
years = ["2017", "2018", "2019", "2020", "2021", "2022"]

gdpDf["mean_gdp"] = gdpDf[years].mean(axis=1)

# Drop Rest
gdpDf = gdpDf.drop(years, axis=1)

### IndustryClassification

In [8]:
# Compress NAICS functions
def map_naics_code(naics_code):
  code_prefix = int(naics_code[:3])

  if code_prefix < 310: 
    return str(code_prefix)[:2]
  else:
    if code_prefix == 321 or ( 327 <= code_prefix <= 339):
      return "321,327-339"
    elif (311 <=code_prefix <= 316) or (322 <= code_prefix <= 326):
      return "311-316,322-326"
    else:
      return "31-33"

# Apply function
occDf["IndustryClassification"] = occDf["naics"].apply(map_naics_code)
patDf["IndustryClassification"] = patDf["naics"].apply(map_naics_code)

### Aggregate Pattern Duplicates

In [9]:
def agg_noise(nf):
  weight = {"G": 1, "H": 2, "J": 3}

  return max(nf, key=lambda letter: weight[letter])

patDf = patDf.groupby(["FIPS", "naics", "IndustryClassification"]).agg({
  "ap_nf": agg_noise,
  "ap": "sum",
  "est": "sum",
}).reset_index()

### Emp Total County NAICS based on Top 5 Occupations

In [10]:
emp_top5 = occDf.groupby(["FIPS", "naics"])["emp_occupation"].sum().reset_index()

In [11]:
emp_top5 = emp_top5.rename({"emp_occupation": "emp_total_county_naics_top5_occ"}, axis=1)

occDf = pd.merge(occDf, emp_top5, on=["FIPS", "naics"])

### Annual Pay per Person in Top 5 OCC

***as*** as in annual salary

In [12]:
masterFrame = pd.merge(occDf, patDf, on=["FIPS", "naics", "IndustryClassification"])

In [13]:
masterFrame["as"] = masterFrame["ap"] / masterFrame["emp_total_county_naics"]
masterFrame["as_top5"] = masterFrame["as"] * masterFrame["emp_total_county_naics_top5_occ"]

## Merging

As occupation and pattern was already merged we just need to add gdp

In [14]:
masterFrame = pd.merge(masterFrame, gdpDf, on=["FIPS", "IndustryClassification"])

## Cleanup

- Drop IndustryClassificationas it was only used for merging
- Reorder columns
- Sort in order of FIPS -> NAICS -> OCC_CODE

In [15]:
masterFrame = masterFrame.drop(["IndustryClassification"], axis=1)

In [16]:
master_order = ["FIPS", "naics", "mean_gdp", "emp_total_county_naics", "emp_total_county_naics_top5_occ", "ap_nf", "ap", "est", "OCC_CODE", "emp_occupation", "as", "as_top5"]
masterFrame = masterFrame[master_order]

In [17]:
masterFrame = masterFrame.sort_values(by=["FIPS", "naics", "OCC_CODE"], ascending=True).reset_index(drop=True)

## Export Master Frame

In [20]:
print(masterFrame)

       FIPS   naics  mean_gdp  emp_total_county_naics  emp_total_county_naics_top5_occ ap_nf     ap  est OCC_CODE  emp_occupation    as  as_top5
0      1111    2381  16958.83                       6                             0.18     J    275    3  49-9071            0.02 45.83     8.33
1      1111    2381  16958.83                       6                             0.18     J    275    3  51-4121            0.16 45.83     8.33
2      1113    2381  52120.17                      28                             0.94     G    851    7  49-9071            0.14 30.39    28.62
3      1113    2381  52120.17                      28                             0.94     G    851    7  51-4031            0.02 30.39    28.62
4      1113    2381  52120.17                      28                             0.94     G    851    7  51-4041            0.01 30.39    28.62
...     ...     ...       ...                     ...                              ...   ...    ...  ...      ...             ... 

In [ ]:
pd.to_pickle(masterFrame, "../data/processed/masterFrame.pkl")